In [1]:
from QuaternionTools import Q_normalize,Q_conj,Q_rotateVector
from QuaternionTools import Q_make, Q_inv, Q_mult
import numpy as np
from EllipsoidFuncs import *  #BLH2XYZ,XYZ2BLH
from RotationMatrixTools import *

# Chaiten Vulkano
# -42.B1=-42.839024° North  und -72.652945° East    H=890m
# Chaiten Town
# B=-42.922505° North und L=-72.729272° East  H=0m

In [2]:
# Coordinates of Chaiten Vulcano
B1=-42.839024
L1=-72.652945
h1=890.0+18.6489
# Chaiten Town will be Origin of local Coordinate-System
B2=-42.922505
L2=-72.729272
h2=0.0+18.1882
#http://geographiclib.sourceforge.net/cgi-bin/GeoidEval?input=-42.922505+-72.729272&option=Submit


x1,y1,z1=BLH2XYZ(B1,L1,h1,"WGS84")
Pv=np.array([[x1],[y1],[z1] ])
print "Chaiten Volcano located at: Lat:{}° Lon:{}° Hight:{}m\n {} \n".format(B1,L1,h1,Pv)


x2,y2,z2=BLH2XYZ(B2,L2,h2,"WGS84")
Pchaiten=np.array([[x2],[y2],[z2] ])
print "Chaiten Town, local Coordinate Origin located at: Lat:{}° Lon:{}° Hight:{}m\n {} \n".format(B2,L2,h2,Pchaiten)

dx21=x1-x2
dy21=y1-y2
dz21=z1-z2

#dx21=x2-x1
#dy21=y2-y1
#dz21=z2-z1

dx=np.array([[dx21],[dy21],[dz21] ])
print "Coordinate diff between the two Point, related to WGS84: \n {}\n".format(dx)

print "Distance of the two points: {} meters".format(np.sqrt(np.dot(dx.T,dx))[0][0])


Chaiten Volcano located at: Lat:-42.839024° Lon:-72.652945° Hight:908.6489m
 [[ 1396813.96580858]
 [-4471715.2539631 ]
 [-4315023.66401024]] 

Chaiten Town, local Coordinate Origin located at: Lat:-42.922505° Lon:-72.729272° Hight:18.1882m
 [[ 1388788.28313284]
 [-4466922.38891514]
 [-4321213.89921993]] 

Coordinate diff between the two Point, related to WGS84: 
 [[ 8025.68267574]
 [-4792.86504795]
 [ 6190.23520969]]

Distance of the two points: 11211.697005 meters


In [3]:
qZ=Q_normalize(Q_make(360+L2,[0,0,1]))  # dreht um Z (R3) um Längengrad L
print qZ
qY=Q_normalize(Q_make((90-B2),[0,1,0]))  # dreht um Y (R2) um Breitengrad (90-B)
print qY
qYZ=Q_mult(qY,qZ) # räumliche Drehachse... kann daraus gerechnet werden
print qYZ

qp1=np.append([0],dx)
pr1=Q_mult(Q_mult(qYZ,qp1),Q_inv(qYZ))
qRotP1   = np.array([[pr1[1]],[pr1[2]],[pr1[3]]])
qRotP1[1]=-qRotP1[1]
qRotP1[2]=-qRotP1[2]

print "\nLocal Coordinates of the Vulcano in the ENU-Observersystem (East,North,Up) at Chaiten Town: \n"
print qRotP1

print np.sqrt(np.dot(qRotP1.T,qRotP1))[0][0]

[-0.80525991  0.          0.          0.59292198]
[ 0.39936916  0.          0.9167902   0.        ]
[-0.32159597  0.54358506 -0.7382544   0.23679475]

Local Coordinates of the Vulcano in the ENU-Observersystem (East,North,Up) at Chaiten Town: 

[[ 6027.12140492]
 [ 9086.76511185]
 [ 2608.95709111]]
11211.697005


In [4]:
# Zenitwinkel
sPS=np.sqrt(np.dot(qRotP1.T,qRotP1))
print "distance Observer to Sat.: ", sPS/1000 , " km"
print "local coordinates of the Sat [x y z] : \n", qRotP1,"\n"

Elev=np.arcsin(qRotP1[2]/sPS)
print "Elevation of the Sat.: ", Elev*180.0/np.pi , "°"

Az=np.arctan2(qRotP1[0],qRotP1[1])   # qRotP1[0] = East ... Y Geodätisch und qRotP1[1] = North ... X geodätisch
#Az=np.arccos(qRotP1[0]/(sPS*np.sin(Elev)))
print "Azimuth of the Sat.  : ",Az*180.0/np.pi, "°"

#das ist eine Links system mit West, South, Down.... daher sind alle Koord... negativ!!!!

# Veranschaulichung
# http://www.geodz.com/deu/d/lokales_ellipsoidisches_Koordinatensystem


distance Observer to Sat.:  [[ 11.211697]]  km
local coordinates of the Sat [x y z] : 
[[ 6027.12140492]
 [ 9086.76511185]
 [ 2608.95709111]] 

Elevation of the Sat.:  [[ 13.4560591]] °
Azimuth of the Sat.  :  [ 33.55574791] °


# Test with Rot-Matrix

In [ ]:
R3L2=Rypr(L2,0,0)
R2B2=Rypr(0,90-B2,0)

dxz=np.dot(R3L2,dx)
print "local coordinates after Rotation around Z-Axis (R3)"
print "dxz.T = \n",dxz, "\n"

dxzy=np.dot(R2B2,dxz)
print "local coordinates after Rotation around Y-Axis(R2)"
print "dxzy.T = \n",dxzy
print "\nApply S1, x-component negative  "
dxzy[0]=-dxzy[0]
print "\nLocal coordinates of the Satellite in the ENU-Observersystem (East,North,Up)"
print "dxzy.T = \n",dxzy


# First Tests

In [ ]:
q1=Q_make(60,[0,0,5])
print q1
q2=Q_conj(q1)
print q2
print Q_normalize(q2)

In [ ]:

#makeQuat(60,[2,3,4])

qE=Q_normalize(Q_make(60,[0,0,-1]))
print "Prüfung: " , np.sqrt(np.dot(qE,qE))

iQE=Q_inv(qE)

Q_rotateVector([0,0,-1],60,[0,2,6])

In [ ]:
p=np.array([0,0,2,6])
pr=Q_mult(Q_mult(qE,p),iQE)
print pr

In [ ]:
arr=np.array([0,2,6])
np.append([0],arr)

# Koordinatenten des Ursprungspunktes des Lok. Zenitsystems

In [ ]:
print "X: ",X, "Y: ",Y,"Z: ",Z, "im WGS84 (geozentrisch)"

In [ ]:
print np.eye(3)[0,0]
S1=np.eye(3)
S1[0,0]=-S1[0,0]
print S1

# Rotationsquaternionen

In [ ]:
qZ=Q_normalize(Q_make(L2,[0,0,1]))  # dreht um Z (R3) um Längengrad L
print qZ

In [ ]:
qY=Q_normalize(Q_make((90-B2),[0,1,0]))  # dreht um Y (R2) um Breitengrad (90-B)
print qY

In [ ]:
qYZ=Q_mult(qZ,qY) # räumliche Drehachse... kann daraus gerechnet werden
print qYZ

In [ ]:
arr1=np.array([dx21,dy21,dz21])
qp1=np.append([0],arr1)

pr1=Q_mult(Q_mult(qYZ,qp1),Q_inv(qYZ))
RotP1   = np.array([pr1[1],pr1[2],pr1[3]])
RotP1[0]=-RotP1[0]
print RotP1
print " das sollten die Koordinaten im lokalen ell. Zenitsystem sein.......bitte validieren!!!"

In [ ]:
#687166 Ostwert  5245799 Nordwert
#692056 Ostwert  5254192 Nordwert

print np.sqrt(pow((692056-687166),2) + pow((5254192-5245799),2))
print 844-15
print np.sqrt(pow(9713.62697451,2) + pow(829,2))


TestL2=Q_rotateVector([0,0,1],L2,[dx21,dy21,dz21])
print TestL2
TestL2B2=Q_rotateVector([0,1,0],(90-B2),TestL2)
print TestL2B2


# Rotationsmatrix R3

In [ ]:
R3=Rypr(L2,0,0)

In [ ]:
R2=Rypr(0,(90-B2),0)

In [ ]:
R3P=R3*np.array([[x1,y1,z1]]).T
#print R3.shape
#print R3P
print "   "
R2R3P1=S1*R2*R3P
print R2R3P1

R3P=R3*np.array([[x2,y2,z2]]).T
#print R3.shape
#print R3P
print "   "
R2R3P2=S1*R2*R3P
print R2R3P2


print "  "
print R2R3P2 -R2R3P1

# Der Punkt P(0, 2, 6) soll um 60° bzgl. der z-Achse rotiert werden. 

In [1]:
w=60
x=0
y=2
z=6

#Rotationsachse = Z-Achse
# --> Quaternion Imaginärteil [0,0,1]  is die Rotationachse um die gedreht wird

#Einheitsquaternion erstellen
q=np.array([ np.cos((60*math.pi)/(2*180)),0,0,-1*np.sin((60*math.pi)/(2*180)) ] )
print q

# Normieren des Quaternions
print np.sqrt(np.dot(q,q))

qE=q/np.sqrt(np.dot(q,q))
print qE

print "Prüfung: " , np.sqrt(np.dot(qE,qE))


## Rotation des Vektors [0,2,6] um 60°
# Erweitern des Vektors auf 4.Dim, einfach 0 als erste Komponente hinzufügen
p=np.array([0,0,2,6])
print p
# inverse von qE bilden, ist gleich dem konjugierten weil Einheitsquaternion
iqE=-qE
iqE[0]=-iqE[0]
print iqE

#ausführen der Rotation als quaternionen Multiplikation
pr=q_mult(q_mult(qE,p),iqE)
print pr

pr=q_mult(qE,q_mult(p,iqE))
print pr

NameError: name 'np' is not defined

In [ ]:
w1, x1, y1, z1 = qE
print w1,x1, y1, z1

In [ ]:
#Rotationsmatrix
def Rypr(y, p, r):
    '''
    Rotationsmatrix für y=yaw, p=pitch, r=roll in degrees
    '''
    # from Degree to Radians
    y = y*np.pi/180.0
    p = p*np.pi/180.0
    r = r*np.pi/180.0
 
    Rr = np.matrix([[1.0, 0.0, 0.0],[0.0, np.cos(r), -np.sin(r)],[0.0, np.sin(r), np.cos(r)]])
    Rp = np.matrix([[np.cos(p), 0.0, np.sin(p)],[0.0, 1.0, 0.0],[-np.sin(p), 0.0, np.cos(p)]])
    Ry = np.matrix([[np.cos(y), -np.sin(y), 0.0],[np.sin(y), np.cos(y), 0.0],[0.0, 0.0, 1.0]])
 
    return Ry*Rp*Rr

In [ ]:
def normQ(q):
    '''Calculates the normalized Quaternion
    a is the real part
    b, c, d are the complex elements'''
    # Source: Buchholz, J. J. (2013). Vorlesungsmanuskript Regelungstechnik und Flugregler.
    # GRIN Verlag. Retrieved from http://www.grin.com/de/e-book/82818/regelungstechnik-und-flugregler
    a, b, c, d = q
 
    # Betrag
    Z = np.sqrt(a**2+b**2+c**2+d**2)
 
    return np.array([a/Z,b/Z,c/Z,d/Z])